# Implent your own Decision Tree/Random Forest!


In this python notebook, you will create a basic decision tree on pandas data, and train a classifier on the Iris dataset. Then, you will implement a type of bagging and create a random forest classifier!

First, import the required modules:

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

Then import and preview the data:

In [ ]:
iris = load_iris()
df = pd.DataFrame(iris.data)
df['species'] = iris.target 
df.head()

We have four features labeled 0, 1, 2, and 3. These stand for the length and the width of the sepals and petals, in centimeters. We want to use these four features to predict whether the species is one of three types of Iris plant, labeled 0, 1, or 2. 

Now, we split the dataset into training and test samples.

In [ ]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
train = train.drop(['is_train'], axis = 1)
test = test.drop(['is_train'], axis = 1)

# Disorder (Splitting Metric)

First, we want to implement some measure of disorder in a set of data.

Implement either information gain or GINI impurity discussed in class. (for reference the equations are in 189 notes here https://www.eecs189.org/static/notes/n25.pdf) 


The argument `data` is a pandas dataframe containing the features and labels of several data points. We calculate disorder based on the labels, or the last column of the data. Note: make sure that you make this function work for different data (i.e. your function should work for data of different dimensions).

In [ ]:
def disorder(data):
    # Using Gini impurity
    if data.empty:
        return 1
    label_counts = data.iloc[:,-1].value_counts()
    classes = label_counts.index
    num_rows = data.shape[0]
    impurity = 1
    for c in classes:
        p_k = label_counts[c] / num_rows
        impurity -= p_k ** 2
    return impurity

We now create a split function. This function takes in a dataset, and indices for a row and column. We then return two dataframes split on the `column`th feature. The left dataset should contain all of the data where the `column`th feature is greater or equal to the `column`th feature of the `row`th datapoint, and the right should contain the rest. Use the disorder metric you implemented in the function above.

In [ ]:
def split_on_row_column(data, row, column):
    split_val = data.iloc[row, column]
    num_rows = data.shape[0]
    left = pd.DataFrame()
    right = pd.DataFrame()
    for i in range(num_rows):
        if (data.iloc[i, column] >= split_val):
            left = left.append(data.iloc[i])
        else:
            right = right.append(data.iloc[i])
    return left, right

We now want to define our recursive tree class. During training, there are two cases for a node. If the data is all one label, the node is a leaf node, and we return this value during inference. If the data is not all the same label, we find the best split of the data by iterating through all of features and rows in the data. Use the split function defined above to find the best split.

Inference takes in a row of a pandas dataframes and returns the predicted class. 

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.feature_index = None
        self.split_value = None
        self.left_child = None
        self.right_child = None
        self.best_gini = 1
        self.predicted_label = None
    
    def find_best_split(self):
        best_gini = 1
        best_feature_index = 0
        best_split_value = 0
        
        num_features = self.data.shape[1] - 1
        num_rows = self.data.shape[0]
        
        for column in range(num_features):
            for row in range(num_rows):  
                left, right = split_on_row_column(self.data, row, column)
                num_rows_left = left.shape[0]
                num_rows_right = right.shape[0]
                
                gini_left = disorder(left)
                gini_right = disorder(right)

                #weighted average of the Gini impurity of the children
                gini = (num_rows_left * gini_left + num_rows_right * gini_right) / num_rows
                if gini < best_gini:
                    best_gini = gini
                    best_feature_index = column
                    best_split_value = self.data.iloc[row, column]
                    best_left = left
                    best_right = right

        return best_gini, best_feature_index, best_split_value, best_left, best_right
    
    def train(self):
        self.best_gini, self.feature_index, self.split_value, left, right = self.find_best_split()
        if disorder(left) == 0:
            left_node = Node(left.copy())
            left_node.predicted_label = left_node.data.iloc[0, -1]
            self.left_child = left_node
        else:
            left_node = Node(left.copy())
            left_node.train()            
            self.left_child = left_node
        if disorder(right) == 0:
            right_node = Node(right.copy())
            right_node.predicted_label = right_node.data.iloc[0, -1]
            self.right_child = right_node
        else:
            right_node = Node(right.copy())
            right_node.train()
            self.right_child = right_node   
        return self
    
    def inference(self, row):
        node = self
        while node.left_child or node.right_child:
            if row[node.feature_index] >= node.split_value:
                node = node.left_child
            else:
                node = node.right_child
        return node.predicted_label

Now initialize and train a decision tree:

In [ ]:
tree = Node(train)
tree.train()

Note that we don't check the training accuracy here (why?). We now want to validate our tree on the test dataset:

In [ ]:
def validate(model, data):
    ct = 0
    corr = 0
    for i in range(test.shape[0]):
        data = test.iloc[i]
        ct += 1
        if model.inference(data) == data['species']:
            corr += 1
    return corr/ct

validate(tree, test)

## Random Forest!

Now we will implement data bagging with a random forest! The set up is similar to a single tree. We pass in the data to the forest, along with hyperparameters `n`, `frac`, anbd `m`, which correspond to the number of trees, the fraction of the dataset to use in each bag, the number or percentage of random features (depending on your own implementation) selected at each possible split. Note that the difference between random forests and just bagging  is that random forests select a random subset of features per bag while bagging assumes all features are present in each sample. A good estimate for m in a dataset with `num_features` is m = sqrt(`num_features`). In the inference step we tally the number of votes from each decision tree and return the label with the most amount of votes.

In [ ]:
class Forest:
    def __init__(self, data, n, frac):
        self.data = data
        self.n = n
        self.frac = frac
        self.trees = []
    
    def train(self):
        for i in range(self.n):
            sample = self.data.sample(int(self.frac * self.data.shape[0]))
            tree = Node(sample).train()
            self.trees.append(tree)
        print(self.trees)
        
    def inference(self, x):
        result = []
        for i in range(self.n):
            result.append(self.trees[i].inference(x))
        return max(set(result), key=result.count)       

Train and validate your forest!

In [ ]:
forest = Forest(train, 30, .5)
forest.train()

In [ ]:
validate(forest, test)